# import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt 

from sklearn import metrics

# load data

In [ ]:
data1 = pd.read_csv('cleandata.csv')
y = data1.account_type.values.tolist()
def extract_features(x):
    columns=['favourites_count','followers_count','friends_count','statuses_count','verified','created']
    # x.loc[:,'created']= creation_year(x['created_at'])
    x.loc[:,'created'] = x.apply(lambda row :creation_year(row["created_at"]), axis=1)
    x=x.loc[:,columns]
    return x

